In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from os import listdir
from os.path import isfile, join
from PIL import Image
from torchvision import datasets, models
from torchvision import transforms as T
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import time as time
import copy
from torchvision.datasets import OxfordIIITPet


In [2]:
path = "/Data"

num_classes = 2 #Number of classes in dataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
image_size = 64
batch_size = 128

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

/home/da150x_47/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [1]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def initialize_model(model_name, num_classes, feature_extracting): #Initialize Resnet

    if model_name == "resnet":

        model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True) 
        
        set_parameter_requires_grad(model, feature_extracting)
        
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs,num_classes) # Update last layer to binary classification (Dog/cat)
        input_size = 224 #"Finally, notice that inception_v3 requires the input size to be (299,299), whereas all of the other models expect (224,224)."
        model = model.to(device)
        
    return model, input_size



def readData(target_type = 'breeds'):
    breed_to_species = {0:0, 1:1, 2:1, 3:1, 4:1, 5:0, 6:0, 7:0, 8:1, 9:0, 10:1,
                    11:0, 12:1, 13:1, 14:1, 15:1, 16:1, 17:1, 18:1, 19:1, 20:0,
                    21:1, 22:1, 23:0, 24:1, 25:1, 26:0, 27:0, 28:1, 29:1, 30:1,
                    31:1, 32:0, 33:0, 34:1, 35:1, 36:1
                   }
    
    transform = T.Compose([T.Resize(image_size),
                          T.CenterCrop(image_size),
                          T.ToTensor()])
    
    if target_type == 'species':
        target_function = (lambda x: breed_to_species.get(x))
    elif target_type == 'breeds':
        target_function = None
    else:
        raise Exception("Wrong target type")
        
    trainval = OxfordIIITPet(path,
                           target_types= "category",
                           transform=transform,
                           split = 'trainval',
                           target_transform = target_function,
                             download = True
                          )
    test = OxfordIIITPet(path,
                           target_types= "category",
                           transform=transform,
                           split = 'test',
                           target_transform = target_function
                        )
                         
    train, val = splitData(trainval, 0.8, 0.2)
    return train, val, test
    
def splitData(dataset,nrTrain,nrVal):
    len1 = int(nrTrain*len(dataset))
    len2 = int(len(dataset) - len1)
    train, val = torch.utils.data.random_split(dataset, [len1,len2])
    return train, val

def show_example(img, label):
    print('Label: ', train_dataset.classes[label], "("+str(label)+")")
    plt.imshow(img.permute(1, 2, 0))




In [4]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.

                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [5]:
def score(model, dataloader):
    running_corrects = 0
    model.eval()
    for inputs, labels in dataloader:
        
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        _, preds = torch.max(outputs, 1)

        running_corrects += torch.sum(preds == labels.data)

    acc = running_corrects.double() / len(dataloader.dataset)
    print('Acc: {:4f}'.format(acc))

In [6]:
train, val, test = readData('breeds')


model,input_size = initialize_model(model_name = "resnet",
                                    num_classes = 2,
                                    feature_extracting = True)


dataloaders = {label: torch.utils.data.DataLoader(data, batch_size=batch_size,
                                         shuffle=True, num_workers=4)
          for label, data in zip(['train', 'val'],[train, val])}

test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size,
                                         shuffle=False, num_workers=4)

optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

best_model, val_acc_history = train_model(model = model,
                                     num_epochs = 10,
                                     dataloaders = dataloaders,
                                     criterion = criterion,
                                     optimizer = optimizer
                                    )


Using cache found in /home/da150x_47/.cache/torch/hub/pytorch_vision_v0.10.0


Epoch 0/9
----------


IndexError: Target 5 is out of bounds.

In [ ]:
score(best_model, test_dataloader)